## AutoAIデータ作成用

### 共通事前処理

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)
# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format
# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)
# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14
# 乱数の種
random_seed = 123

#### データ読み込み

In [ ]:
# 公開データのダウンロードと解凍
!wget https://archive.ics.uci.edu/ml/\
machine-learning-databases/00222/bank.zip -O bank.zip
!unzip -o bank.zip

# bank-full.csvをデータフレームに取り込み
df_all = pd.read_csv('bank-full.csv', sep=';')

# 項目名を日本語に置き換える
columns = [
    '年齢', '職業', '婚姻', '学歴', '債務不履行', '平均残高',
    '住宅ローン', '個人ローン', '連絡手段', '最終通話日',
    '最終通話月', '最終通話秒数', '通話回数_販促中',
    '前回販促後_経過日数', '通話回数_販促前', '前回販促結果',
    '今回販促結果'
]
df_all.columns = columns
y = df_all['今回販促結果'].values


#### データ分割

In [ ]:
# 訓練データと検証データの分割
# 訓練データ60% 検証データ40%の比率で分割する
test_size = 0.4

from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(
  df_all, test_size=test_size, random_state=random_seed,
  stratify=y)

#### 結果確認

In [ ]:
display(df_train.head())

In [ ]:
display(df_test.head())

#### 保存

In [ ]:
fn_train = 'bank-train-autoai.csv'
df_train.to_csv(fn_train, index=False)

In [ ]:
fn_test = 'bank-test-autoai.csv'
df_test.to_csv(fn_test, index=False)

#### Google Colabからダウンロード

In [ ]:
try:
    from google.colab import files
    files.download(fn_train) 
    files.download(fn_test)      
except:
    print('not google colab')

### 検証データをJSON形式に変更

In [ ]:
df_test_yes = df_test[df_test['今回販促結果']=='yes'].head(20)
df_test_no = df_test[df_test['今回販促結果']=='no'].head(20)
display(df_test_yes.head())
display(df_test_no.head())

In [ ]:
x_test_yes = df_test_yes.drop(['今回販促結果'], axis=1)
x_test_no = df_test_no.drop(['今回販促結果'], axis=1)
display(x_test_yes.head())
display(x_test_no.head())

In [ ]:
columns = list(x_test_yes.columns)
yes_values = x_test_yes.values
no_values = x_test_no.values
print(columns)
print(yes_values[:5,:])

In [ ]:
# yes用jsonデータ
import json
w1 = {}
w1['fields'] = columns
w1['values'] = yes_values.tolist()
u1 = {"input_data": [w1]}
d1 = json.dumps(u1,ensure_ascii=False)
print(d1)

In [ ]:
# no用jsonデータ
w2 = {}
w2['fields'] = columns
w2['values'] = no_values.tolist()
u2 = {"input_data": [w2]}
d2 = json.dumps(u2, ensure_ascii=False)
print(d2)